In [102]:
import pandas as pd
import csv
import random

random.seed(3500)

In [103]:
# UTILITY FUNCTIONS
def make_quote(l, idx):
    return '"' + l[idx] + '"'

def make_write_str(l, join_str=',\n', double_new_line=True):
    ret = '\n(' + join_str.join(l) + ')'
    if double_new_line:
        return ret + '\n'
    return ret

def make_real_list(l):
    l = l[1:-1] # removes the '[' and ']'
    return l.split(', ')

In [104]:
# Universities
with open('inserts/university_inserts.sql', 'a') as write_file,\
     open('data/universities.csv', 'r') as read_file:
    reader = csv.reader(read_file)
    
    next(reader) # Remove header
    
    uni_dict = {}
    id_counter = 0
    write_file.write('INSERT INTO University (university_id, university_name) VALUES\n')
    
    line_num = 0
    
    for line in reader:
        if line[4] != '-' and line[4] not in uni_dict:
            
            if line_num != 0 and line_num != 2870:
                write_file.write(',\n')
            line_num += 1
            
            uni_dict[line[4]] = id_counter
            write_file.write('\t({0}, "{1}")'.format(id_counter, line[4]))
            id_counter += 1
            
    write_file.write(';')

In [105]:
# Create Users
with open('inserts/user_inserts.sql', 'a') as write_file, \
     open('data/users.csv', 'r') as read_file:
    
    reader = csv.reader(read_file)
    ins_str = """INSERT INTO User 
        (first_name,
        last_name, 
        username, 
        email, 
        password,  
        university_id) VALUES"""
    write_file.write(ins_str)
    
    next(read_file) # skip header
    
    line_num = 0
    
    for line in reader:
        
        if line_num != 0 and line_num != 101:
            write

In [106]:
# Create Users
with open('inserts/user_inserts.sql', 'a') as write_file, \
     open('data/users.csv', 'r') as read_file:
    
    reader = csv.reader(read_file)
    ins_str = """INSERT INTO User 
        (first_name,
        last_name, 
        username, 
        email, 
        password,  
        university_id) VALUES"""
    write_file.write(ins_str)
    
    next(read_file) # skip header
    
    line_num = 0
    
    for line in reader:
        
        if line_num != 0 and line_num != 101:
            write_file.write(',')
        line_num += 1
        
        line.pop(0)
        line = [make_quote(line, i) for i in range(len(line))]
        line.append(str(random.randint(0, 2869)))
        write_str = make_write_str(line, join_str=',', double_new_line=False)
        
        write_file.write(write_str)
    
    write_file.write(';')
    

In [107]:
#RECIPES
with open('data/food_network_recipes_1_with_description.csv', 'r') as read_file,\
     open('inserts/recipe_inserts.sql', 'a+') as write_file:
    reader = csv.reader(read_file)
    
    ins_str = '''INSERT INTO Recipe 
    (recipe_id, 
    title,
    description,
    minutes, 
    servings, 
    instructions, 
    user_id) VALUES'''
    write_file.write(ins_str)
    
    next(reader)
    
    who_wrote_what = []
    
    line_num = 0
    
    for line in reader:
        
        if line_num != 0 and line_num != 44:
            write_file.write(',\n')
        line_num += 1
        
        line[1] = make_quote(line, 1)
        line[2] = make_quote(line, 2)
        line[6] = make_quote(line, 6)
        line.pop(3)
        creator_id = random.randint(0, 106)
        line.append(str(creator_id))
        write_str = make_write_str(line, double_new_line=False)
        write_file.write(write_str)
        
        # We need this later when we make ratings and saves
        # to make sure people aren't rating a saving their own recipes
        who_wrote_what.append((line[0], creator_id))

    write_file.write(';')
            

In [108]:
# INGREDIENTS AND RECIPE INGREDIENTS
with open('data/food_network_ingredients_1.csv', 'r') as read_file2, \
     open('inserts/ingredient_inserts.sql', 'a+') as ing_file, \
     open('inserts/ri_inserts.sql', 'a+') as ri_file:
    reader = csv.reader(read_file2)
    ing_file.write('INSERT INTO Ingredient (ingredient_name)')
    ri_file.write('INSERT INTO Recipe_ingredient (ingredient_id, recipe_id, amount, unit, optional)')
    next(reader)
    
    ing_id_counter = 0
    ing_dict = {}
    
    first_ing = True
    first_ri = True
    
    
    for line in reader:
        recipe_id = line[0]
        ing_list = make_real_list(line[1])
        amount_list = make_real_list(line[2])
        unit_list = make_real_list(line[3])
        option_list = make_real_list(line[4])
        
        for i in range(len(ing_list)):
            if ing_list[i] not in ing_dict:
                
                if not first_ing:
                    ing_file.write(',')
                else:
                    first_ing = False
                    
                ing_file.write('\n(' + ing_list[i] + ')')
                ing_dict[ing_list[i]] = ing_id_counter
                ing_id = str(ing_id_counter)
                ing_id_counter += 1
            else:
                ing_id = str(ing_dict[ing_list[i]])
                
            if not first_ri:
                ri_file.write(',')
            else:
                first_ri = False
                
            write_str = make_write_str([ing_id, recipe_id, amount_list[i], unit_list[i], option_list[i]], join_str=', ', double_new_line=False)
            ri_file.write(write_str)
            
    ri_file.write(';')
    ing_file.write(';')
        
    
    

In [109]:
# Tags and recipe tags
with open('data/food_network_tags_1.csv', 'r') as tags_file,\
     open('inserts/tags_inserts.sql', 'a') as t_write,\
     open('inserts/recipe_tag_inserts.sql', 'a') as rt_write:
    reader = csv.reader(tags_file)
    
    tag_id_counter = 0
    tag_id_dict = {}
    
    tag_ins_str = 'INSERT INTO Tag (tag_id, tag_name) VALUES \n'
    rt_ins_str = 'INSERT INTO Recipe_tag (tag_id, recipe_id) VALUE \n'
    
    t_write.write(tag_ins_str)
    rt_write.write(rt_ins_str)
    
    next(reader)
    
    first_tag = True
    first_rt = True
    
    for line in reader:
        tags = make_real_list(line[1])
        for tag in tags:
            if tag not in tag_id_dict:
                
                if not first_tag:
                    t_write.write(',')
                else:
                    first_tag = False
                                
                tag_id_dict[tag] = tag_id_counter
                t_write.write('\n\t({}, "{}")'.format(tag_id_counter, tag))
                tag_id_counter += 1
            
            if not first_rt:
                rt_write.write(',')
            else:
                first_rt = False
            
            rt_write.write('\n\t({}, {})'.format(tag_id_dict[tag], line[0]))
                
    t_write.write(';')
    rt_write.write(';')
        
    

In [110]:
# appliances and recipe appliances
with open('data/food_network_appliances_1.csv', 'r') as tags_file,\
     open('inserts/appliance_inserts.sql', 'a') as a_write,\
     open('inserts/recipe_appliance_inserts.sql', 'a') as ra_write:
    reader = csv.reader(tags_file)
    
    app_id_counter = 0
    app_id_dict = {}
    
    app_ins_str = 'INSERT INTO Tag (appliance_id, appliance_name) VALUES \n'
    rt_ins_str = 'INSERT INTO Recipe_tag (appliance_id, recipe_id) VALUE \n'
    
    a_write.write(tag_ins_str)
    ra_write.write(rt_ins_str)
    
    next(reader)
    
    first_app = True
    first_ra = True
    
    for line in reader:
        appliances = make_real_list(line[1])
        for app in appliances:
            if app not in app_id_dict:
                
                if not first_tag:
                    a_write.write(',\n')
                else:
                    first_a[[]] = False
                
                app_id_dict[app] = app_id_counter
                a_write.write('\t({}, "{}")'.format(app_id_counter, app))
                app_id_counter += 1
            
            
            if not first_ra:
                ra_write.write(',\n')
            else:
                first_ra = False
            
            ra_write.write('\t({}, {})'.format(app_id_dict[app], line[0]))
                
    a_write.write(';')
    ra_write.write(';')
        


In [111]:
# Follows

with open('inserts/follow_inserts.sql', 'a') as wfile:
    
    existing_follows = []
    
    
    while len(existing_follows) < 100:
        follower = random.randint(0, 105)
        followee = random.randint(0, 105)
        
        if follower!= followee and [follower, followee] not in existing_follows:
            existing_follows.append([follower, followee])
            
    wfile.write('INSERT INTO Follow (follower_id, followee_id) VALUES')        
    
    first_line = True
    
    for pair in existing_follows:
        
        if not first_line:
            wfile.write(',\n')
        else:
            first_line = False
        
        wfile.write('\t({}, {})'.format(pair[0], pair[1]))
        
    wfile.write(';')

In [112]:
# Ratings
with open('inserts/ratings_inserts.sql', 'a') as rfile,\
     open('data/rating_dates.csv', 'r') as date_file:
    reader = csv.reader(date_file)
    
    rfile.write('INSERT INTO Rating (recipe_id, user_id, rating, rating_date) VALUES\n')
    
    line_num = 0
    
    for line in reader:
        pair = []
        while len(pair) == 0:
            recipe = random.randint(0, 43)
            user = random.randint(0, 105)
            if (recipe, user) not in who_wrote_what:
                pair = [recipe, user]
        
        rating = random.randint(0, 10)
        
        if line_num != 0 and line_num != 100:
            rfile.write(',\n')
        line_num += 1
        
        rfile.write("\t({}, {}, {}, '{}')".format(pair[0], pair[1], rating, line[0]))
        
        
    rfile.write(';')
    
    
    
    